In [1]:
import requests
import os
import sys
import pandas as pd
import datetime
import numpy as np

SERVER_HOST = 'http://54.163.150.6:8002'
TOKEN_PATH = '/shmain/model/token'
DATA_PATH = '/shmain/model/data'
CSV_PATH = '/tmp/csv'

In [2]:
def request_tokens():
    """
    Request token list from wristband server

    Input
    ---
        None

    Output
    ---
        [str] List of token

    """
    response = requests.get(SERVER_HOST + TOKEN_PATH)
    tokens = response.json()
    return tokens

In [3]:
token_list = request_tokens()
print(token_list)

['27538649', '82437506']


In [13]:
def save_csv(tokens:[str]):
    """
    Create directory store wristband CSV files

    Input
    ---
        tokens:[str] List of token

    Output
    ---
        None

    """
    # create directory is not exists
    if os.path.isdir(CSV_PATH) == False:
        os.mkdir(CSV_PATH)

    for item in tokens:
        response = requests.get(f'{SERVER_HOST}{DATA_PATH}?token={item}')
        with open(f'{CSV_PATH}/{item}.csv', 'w') as csv_file:
            csv_file.write(response.text)
            csv_file.close()

In [14]:
save_csv(token_list)
print(os.listdir(CSV_PATH))

['27538649.csv', '82437506.csv']


# Data Processing

In [6]:
def clean_disconti(array, time): 
    clean_arr = np.empty((len(array)))
    for i in range(len(array)):
        if (i>2) & (i<len(array)-2):
            if min(abs(array[i]-array[i+1])/(time[i+1]-time[i]).total_seconds(), abs(array[i]-array[i-1])/(time[i]-time[i-1]).total_seconds()) > 0.3:
                clean_arr[i] = (array[i-1]+array[i+1])/2
            else:
                clean_arr[i] = array[i]
        else:
            clean_arr[i] = array[i]
    return clean_arr

function clean_disconti( tempeaature, time)

temperature : ndarray

time : ndarray

output : 修掉不正常的點

In [7]:
def mv_avg(x, window_size):
    output_arr = []
    for data_idx in range(0, len(x)):
        if (data_idx+window_size)>=len(x)-1:
            output_arr.append(np.mean(x[data_idx:]))
        else:
            output_arr.append(np.mean(x[data_idx:data_idx+window_size]))
    return np.array(output_arr)

function mv_avg ( temperature, window_size)

temperature : ndarray

window_size :　int

output : 移動平均，光滑的訊號


In [8]:
def find_conti(x, time):
    start = 0
    while x[len(x)-start-1]-x[len(x)-start-2] > 0:
        start += 1
    return time[-start-1], x[len(x)-1]-x[len(x)-start-1]

function find_conti(temperature)

temperature : ndarray
    
return 開始上升的時間點(距離現在前幾秒) , 在這個區間裡面上升幾度

In [9]:
def data_extraction(path):

    data = pd.read_csv(path)
    data.columns = ["ID", "date", "time", 'hr', 'temp', 'activity']
    data['time'] = data['date']+' '+data['time']
    data = data.drop(['date'], axis = 1)

    data_time = np.array([datetime.datetime.strptime(i, '%Y-%m-%d %H:%M:%S') for i in data['time'].values])
    data_temp = np.array((data['temp'].values)/100)
    modify_temp_watch = clean_disconti(np.copy(data_temp), np.copy(data_time))
    removable_disconti = [i for i, x in enumerate(((modify_temp_watch-data_temp)!=0).astype(int)) if x != 0]

    return data_time, modify_temp_watch, removable_disconti

function data_extraction(path)

return 

data_time : ndarray (時間tag, 陣列)

modify_temp_watch : ndarray (體溫)

removable_disconti : ndarray (有問題點的index)

In [10]:
def main(path, cri_temperature, cri_increasing_time, window_size=None):

    data_time, data_temperature, index_of_remove = data_extraction(path)
    if window_size == None:
        smooth_data = mv_avg(data_temperature, 4)
    else:
        smooth_data = mv_avg(data_temperature, window_size)

    increasing_ti, increasing_temp = find_conti(smooth_data[:-2], data_time)
    if (smooth_data[-1] > cri_temperature) & (data_time[-1] - increasing_ti > datetime.timedelta(seconds = cri_increasing_time)) :
        return 1
    else:
        return 0

function main(data_path, cri_temperature, cri_increasing_time, window_size)

data_path : format = csv

cri_temperature : 高於多少溫度

cri_increasing_time : 升高多少時間(seconds)

window_size : optional default = 4 移動平均採樣點數

return : int 

1:警示
0:沒事

In [15]:
main(CSV_PATH + '/' + token_list[1] +'.csv' , 26, 1)

1